In [24]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "api_key",
    model_name = "llama-3.1-70b-versatile"
)

In [25]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jobs.nike.com/job/R-38694?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Entry - ML Engineer (Remote Work Option)

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsEntry - ML Engineer (Remote Work Option)Beaverton, OregonBecome a Part

In [29]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)
chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input = {'page_data' : page_data})
print(res.content)

```json
{
  "role": "Entry - ML Engineer (Remote Work Option)",
  "experience": "One year of experience preferred",
  "skills": [
    "Machine Learning",
    "Software Engineering",
    "Python",
    "SQL",
    "ETL",
    "Scikit-learn",
    "Dask",
    "TensorFlow",
    "Kubeflow",
    "Spark",
    "EMR",
    "Data Science",
    "AI",
    "Generative AI"
  ],
  "description": "We are actively seeking to hire a Entry Level Machine Learning Engineers to join our AI/ML team. As a Machine Learning Engineer within the AI/ML team, you will be developing advanced analytics systems that directly impact our business. You will work on a cross-disciplinary team (data/API/infra/infosec/ML) to enable data-driven decision making across multiple organizations."
}
```


In [30]:
type(res.content)

str

In [31]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Entry - ML Engineer (Remote Work Option)',
 'experience': 'One year of experience preferred',
 'skills': ['Machine Learning',
  'Software Engineering',
  'Python',
  'SQL',
  'ETL',
  'Scikit-learn',
  'Dask',
  'TensorFlow',
  'Kubeflow',
  'Spark',
  'EMR',
  'Data Science',
  'AI',
  'Generative AI'],
 'description': 'We are actively seeking to hire a Entry Level Machine Learning Engineers to join our AI/ML team. As a Machine Learning Engineer within the AI/ML team, you will be developing advanced analytics systems that directly impact our business. You will work on a cross-disciplinary team (data/API/infra/infosec/ML) to enable data-driven decision making across multiple organizations.'}

In [32]:
type(json_res)

dict

In [33]:
import pandas as pd 
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [34]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [35]:
job = json_res
job['skills']

['Machine Learning',
 'Software Engineering',
 'Python',
 'SQL',
 'ETL',
 'Scikit-learn',
 'Dask',
 'TensorFlow',
 'Kubeflow',
 'Spark',
 'EMR',
 'Data Science',
 'AI',
 'Generative AI']

In [36]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portf

In [37]:
job

{'role': 'Entry - ML Engineer (Remote Work Option)',
 'experience': 'One year of experience preferred',
 'skills': ['Machine Learning',
  'Software Engineering',
  'Python',
  'SQL',
  'ETL',
  'Scikit-learn',
  'Dask',
  'TensorFlow',
  'Kubeflow',
  'Spark',
  'EMR',
  'Data Science',
  'AI',
  'Generative AI'],
 'description': 'We are actively seeking to hire a Entry Level Machine Learning Engineers to join our AI/ML team. As a Machine Learning Engineer within the AI/ML team, you will be developing advanced analytics systems that directly impact our business. You will work on a cross-disciplinary team (data/API/infra/infosec/ML) to enable data-driven decision making across multiple organizations.'}

In [40]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Harish, a business development executive at Firstriik. Firstriik is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Firstriik 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Harish, BDE at Firstriik. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Machine Learning Engineers for Your AI/ML Team

Dear Hiring Manager,

I came across your job posting for an Entry-Level Machine Learning Engineer, and I'm excited to introduce you to Firstriik, an AI & Software Consulting company dedicated to facilitating the seamless integration of business processes through automated tools.

Our team of expert Machine Learning Engineers has extensive experience in developing advanced analytics systems that drive business growth. We've empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

Our capabilities align perfectly with your requirements, including:

* Machine Learning: We've developed and deployed multiple ML models using Scikit-learn, TensorFlow, and other frameworks.
* Software Engineering: Our team has expertise in designing and developing scalable software systems using Python, SQL, and other technologies.
* ETL: We've worked o